In [3]:
import torch
import torchvision
from torch.utils.data import DataLoader
import torch.nn as nn

In [8]:
train_data=torchvision.datasets.MNIST('/data',
                                      train=True,
                                      download=True,
                                      transform=torchvision.transforms.ToTensor())
test_data=torchvision.datasets.MNIST('/data',
                                      train=False,
                                      download=True,
                                      transform=torchvision.transforms.ToTensor())

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 11.4MB/s]


Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 342kB/s]


Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.16MB/s]


Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 6.31MB/s]


Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/MNIST/raw



In [10]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
train_loader=DataLoader(train_data,batch_size=64,shuffle=True,num_workers=4) #Dataloader library makes batch sizes
test_loader=DataLoader(test_data,batch_size=64,shuffle=True,num_workers=4)

cpu


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [12]:
model=nn.Sequential(
    nn.Flatten(), #flatten input
    nn.Linear(784,512),
    nn.ReLU(),
    nn.Linear(512,256),
    nn.ReLU(),
    nn.Linear(256,64),
    nn.ReLU(),
    nn.Linear(64,10) #output layer #no activation, logits=true
)

In [13]:
model=torch.compile(model).to(device)
#compiler just initialises and optimises weight matrices.

In [26]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [15]:
def batch_accuracy(output,label,N):
  y_pred= torch.argmax(output,dim=1)
  correct=(y_pred==label).sum().item()
  return correct/N


In [27]:
def training():
  loss=0
  accuracy=0
  model.train()
  for image,label in train_loader:
    image=image.to(device)
    label=label.to(device)
    output=model(image)
    batch_loss=loss_fn(output,label)
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    accuracy+=batch_accuracy(output,label,label.shape[0])
  print(f'Training accuracy:({accuracy/len(train_loader)})')
  print(f'Training loss:({loss/len(train_loader)})')

In [29]:
def evaluate():
  loss = 0
  accuracy = 0  # Initialize accuracy to 0 instead of model.eval()
  model.eval()  # Call model.eval() to set the model to evaluation mode
  with torch.no_grad():
    for image, label in test_loader:
      image = image.to(device)
      label = label.to(device)
      output = model(image)
      batch_loss = loss_fn(output, label)
      loss += batch_loss.item()
      accuracy += batch_accuracy(output, label, label.shape[0])
  print(f'Test accuracy:({accuracy/len(test_loader)})')

In [30]:
epochs=10
for i in range(epochs):
  print(f'Epoch:{i+1}')
  training()
  evaluate()

Epoch:1


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Training accuracy:(0.9705656982942431)
Training loss:(0.0)
Test accuracy:(0.9704418789808917)
Epoch:2
Training accuracy:(0.9793276918976546)
Training loss:(0.0)
Test accuracy:(0.9768113057324841)
Epoch:3
Training accuracy:(0.9849413646055437)
Training loss:(0.0)
Test accuracy:(0.977109872611465)
Epoch:4
Training accuracy:(0.9880730277185501)
Training loss:(0.0)
Test accuracy:(0.9766122611464968)
Epoch:5
Training accuracy:(0.9899886727078892)
Training loss:(0.0)
Test accuracy:(0.9800955414012739)
Epoch:6
Training accuracy:(0.9924040511727079)
Training loss:(0.0)
Test accuracy:(0.9793988853503185)
Epoch:7
Training accuracy:(0.9929204424307037)
Training loss:(0.0)
Test accuracy:(0.9783041401273885)
Epoch:8
Training accuracy:(0.9936200692963753)
Training loss:(0.0)
Test accuracy:(0.9784036624203821)
Epoch:9
Training accuracy:(0.9950859541577826)
Training loss:(0.0)
Test accuracy:(0.9782046178343949)
Epoch:10
Training accuracy:(0.994186433901919)
Training loss:(0.0)
Test accuracy:(0.9810907